In [2]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [3]:
from src.RAGTraining.TrainingDataGenerator import TrainingDataGenerator

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [4]:
trainingGenerator = TrainingDataGenerator(model_id = 'teknium/OpenHermes-2.5-Mistral-7B', queries_df_path='insight_user_db_publarticles_search_audit.csv', responses_file_path='gpt-responses.csv')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [80]:
trainingGenerator.content_tokens

6342

2024-02-26 14:47:30,755| ERROR   | Could not establish connection from local ('127.0.0.1', 3095) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-26 14:47:40,836| ERROR   | Could not establish connection from local ('127.0.0.1', 3095) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-26 14:47:50,910| ERROR   | Could not establish connection from local ('127.0.0.1', 3095) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-26 14:48:00,997| ERROR   | Could not establish connection from local ('127.0.0.1', 3095) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ss

In [5]:
import json

with open('training_article_ids_latest.json', 'r') as f:
    best_results = json.load(f)
trainingGenerator.best_results = best_results

# with open('training_user_prompts_latest.json', 'r') as f:
#     user_prompts = json.load(f)
# trainingGenerator.user_prompts = user_prompts

In [6]:
len(trainingGenerator.user_prompts)

0

In [7]:
trainingGenerator.generate_prompts_for_queries()
trainingGenerator.save_results_to_json(trainingGenerator.user_prompts, 'training_user_prompts_latest.json')

  0%|          | 0/575 [00:00<?, ?it/s]

In [10]:
# # trainingGenerator.get_best_articles_for_queries(max_num_random_dates=3, randomize_num_dates_per_query=True)

# len(trainingGenerator.best_results)

# # trainingGenerator.save_results_to_json(trainingGenerator.best_results, 'training_article_ids_latest.json')

# trainingGenerator.generate_prompts_for_queries()

# trainingGenerator.save_results_to_json(trainingGenerator.user_prompts, 'training_user_prompts_latest.json')

In [18]:
trainingGenerator.get_gpt4_attributes_for_article_ids(fraction=1)

completed 50 in 61.920973777770996 seconds
completed 100 in 123.80046486854553 seconds
completed 150 in 189.57392477989197 seconds
completed 200 in 256.99304699897766 seconds
completed 250 in 329.7156319618225 seconds
completed 300 in 395.3889470100403 seconds
completed 350 in 459.97045397758484 seconds
completed 400 in 517.1722056865692 seconds
completed 450 in 593.3677561283112 seconds
completed 500 in 664.4186198711395 seconds
completed 550 in 757.2213108539581 seconds
completed 600 in 820.6438851356506 seconds
completed 650 in 899.900661945343 seconds
completed 700 in 995.1711719036102 seconds
completed 750 in 1069.776879787445 seconds
completed 800 in 1153.1405110359192 seconds
completed 850 in 1233.0429251194 seconds
completed 900 in 1315.6264090538025 seconds
completed 950 in 1392.9333128929138 seconds
completed 1000 in 1467.6860070228577 seconds
completed 1050 in 1538.7029638290405 seconds
completed 1100 in 1621.6014659404755 seconds
done in 1664.8576078414917 seconds


1122

In [5]:
import pandas as pd

In [6]:
responses_df = pd.read_csv('gpt-responses.csv',  header=None)

In [7]:
def has_hash(x):
    return '##' in x

In [8]:
responses_df['hash'] = responses_df[6].apply(lambda x: has_hash(x))

In [9]:
import re

def replace_br_tags(text):
    # This pattern matches sequences of more than two <br> tags.
    # The pattern looks for <br> followed by one or more repetitions of <br>, capturing as many as it finds.
    pattern = r'(<br>\s*){3,}'
    # Replacement string with exactly two <br> tags.
    replacement = r'<br><br>'
    # Use re.sub to replace sequences of more than two <br> tags with exactly two.
    modified_text = re.sub(pattern, replacement, text)
    return modified_text

In [10]:
def clean_response(x):
    x = x.replace('\\n','<br>')
    x = x.replace('\n', '<br>')
    x = x.replace('####', '**')
    x = x.replace('###', '**')
    x = x.replace('##', '**')
    x = x.replace('#', '**')
    x = replace_br_tags(x)
    return x

In [11]:
import json

In [12]:
responses_df['cleaned_response'] = responses_df[6].apply(lambda x: json.loads(clean_response(x)))

In [14]:
responses_df.iloc[1003].cleaned_response

"**Understanding Taxes: A Guide to Direct and Indirect Taxes in India**<br><br>**Introduction**<br><br>Taxes form the backbone of a nation's economy, funding various public services and government operations. In India, taxes can be broadly classified into two categories: direct taxes and indirect taxes. This guide aims to simplify these concepts, providing clarity on what they entail and their significance.<br><br>**Direct Taxes**<br><br>Direct taxes are imposed directly on the taxpayer and cannot be shifted to another person or entity. They are levied based on the income or profits of individuals or organizations. Common examples include Income Tax, Corporate Tax, and Capital Gains Tax. The burden of these taxes falls solely on the individual or entity earning the income.<br><br>**Indirect Taxes**<br><br>Indirect taxes are levied on goods and services rather than on income or profits. These taxes are initially paid by the producer or service provider but are ultimately passed on to th

In [15]:
responses_df = responses_df.rename(columns = {1: 'query', 2: 'query_date', 3: 'system_prompt', 4: 'user_prompt', 5: 'model', 6: 'model_response', 7: 'input_tokens', 8: 'output_tokens'})

In [16]:
responses_df

,0,query,query_date,system_prompt,user_prompt,model,model_response,input_tokens,output_tokens,hash,cleaned_response
0,i,Is fd a destroyer,2024-02-20T18:53:33,As the chief editor for an Indian finance and ...,|| user_query: Is fd a destroyer || today dat...,gpt-4-0125-preview,"""I'm sorry for any confusion, but it seems the...",3015,153,False,"I'm sorry for any confusion, but it seems ther..."
1,i,Best credidt cardw for beginer,2024-01-18T08:23:49,As the chief editor for an Indian finance and ...,|| user_query: Best credidt cardw for beginer...,gpt-4-0125-preview,"""**Top 5 Beginner-Friendly Credit Cards: Makin...",5275,346,False,**Top 5 Beginner-Friendly Credit Cards: Making...
2,i,Wint blogs,2024-02-06T19:50:01,As the chief editor for an Indian finance and ...,|| user_query: Wint blogs || today date: 06 F...,gpt-4-0125-preview,"""**A Closer Look at Wint Wealth's Expanding Po...",2472,371,False,**A Closer Look at Wint Wealth's Expanding Por...
3,i,Shashi Tharoor,2024-02-11T11:14:16,As the chief editor for an Indian finance and ...,|| user_query: Shashi Tharoor || today date: ...,gpt-4-0125-preview,"""**Shashi Tharoor's Key Highlights on India-Ca...",2420,360,False,**Shashi Tharoor's Key Highlights on India-Can...
4,i,what's your name,2024-02-27T03:27:57,As the chief editor for an Indian finance and ...,|| user_query: what's your name || today dat...,gpt-4-0125-preview,"""I'm here to assist you with finance and busin...",3714,23,False,I'm here to assist you with finance and busine...
...,...,...,...,...,...,...,...,...,...,...,...
1117,i,how to save for retirement india,2024-02-07T13:06:49,As the chief editor for an Indian finance and ...,|| user_query: how to save for retirement ind...,gpt-4-0125-preview,"""**Step-by-Step Guide to Saving for Retirement...",5586,337,False,**Step-by-Step Guide to Saving for Retirement ...
1118,i,Home loan,2024-01-02T00:09:56,As the chief editor for an Indian finance and ...,|| user_query: Home loan || today date: 02 Ja...,gpt-4-0125-preview,"""**Choosing the Best Home Loan in India: A Com...",3866,619,False,**Choosing the Best Home Loan in India: A Comp...
1119,i,Oil prices,2024-01-21T10:39:18,As the chief editor for an Indian finance and ...,|| user_query: Oil prices || today date: 21 ...,gpt-4-0125-preview,"""**Understanding the Dynamics of Oil Prices: A...",3188,506,False,**Understanding the Dynamics of Oil Prices: A ...
1120,i,plan for marriage,2024-01-28T03:21:25,As the chief editor for an Indian finance and ...,|| user_query: plan for marriage || today da...,gpt-4-0125-preview,"""**A Comprehensive Financial Strategy for Your...",5470,694,False,**A Comprehensive Financial Strategy for Your ...


In [31]:
responses_df = responses_df.drop(columns = [0, 'hash', 'has_slash'])

In [32]:
responses_df.to_csv('gpt-4-cleaned_responses_v1.csv',index=False)

In [54]:
import re

def add_space_after_unicode_representation(text):
    # Pattern to find any Unicode representation (e.g., \u20b9 or \U000020b9) followed directly by digits.
    # This pattern matches both short (\uxxxx) and long (\Uxxxxxxxx) Unicode representations.
    pattern = r'(\\u[0-9a-fA-F]{4}|\\U[0-9a-fA-F]{8})(\d)'

    # Replacement pattern with a space between the Unicode representation and the number.
    replacement = r'\1 \2'

    # Use re.sub to replace the pattern with our desired format.
    modified_text = re.sub(pattern, replacement, text)

    return modified_text

In [47]:
import json

In [70]:
(responses_df[7].sum() * 0.01)/1000 + (responses_df[8].sum() * 0.03)/1000

54.32432